In [1]:
import tensorflow as tf

checkpoint

MyModel.data-00000-of-00001

MyModel.index

MyModel.meta

In [3]:
w1=tf.Variable(tf.random_normal(shape=[2]), name='w1')
w2=tf.Variable(tf.random_normal(shape=[5]), name='w2')

saver = tf.train.Saver()
sess = tf.Session()

sess.run(tf.global_variables_initializer())
saver.save(sess, './checkpoint_dir/MyModel')

'./MyModel'

Another way to save model

# save model every 1000 step
saver.save(sess, './MyModel', global_step = 1000)
# don't save graph 
saver.save(sess, './MyModel', global_step=1000, write_meta_graph=False)
# save model every 2 hours
saver = tf.train.Saver(max_to_keep=5, keep_checkpoint_every_n_hours=2)

# Load Model

In [5]:
# load the graph
saver = tf.train.import_meta_graph('./checkpoint_dir/Mymodel.meta')

In [11]:
# load the parameter
with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph('./checkpoint_dir/MyModel.meta')
    new_saver.restore(sess, tf.train.latest_checkpoint('./checkpoint_dir'))
#     w2:0 is the name of tensor
    print(sess.run('w2:0'))

INFO:tensorflow:Restoring parameters from ./checkpoint_dir\MyModel
[-0.91924554 -0.3694038   0.14456846 -0.7165791   0.27498174]


# Use the restore model

In [1]:
import tensorflow as tf


w1 = tf.placeholder("float", name="w1")
w2 = tf.placeholder("float", name="w2")
b1= tf.Variable(2.0,name="bias") 

#定义一个op，用于后面恢复
w3 = tf.add(w1,w2)
w4 = tf.multiply(w3,b1,name="op_to_restore")
sess = tf.Session()
sess.run(tf.global_variables_initializer())

#创建一个Saver对象，用于保存所有变量
saver_ = tf.train.Saver()

#通过传入数据，执行op
print(sess.run(w4,feed_dict ={w1:4,w2:8}))
#打印 24.0 ==>(w1+w2)*b1

#现在保存模型
saver_.save(sess, './checkpoint_dir/MyModel',global_step=1000)



24.0


'./checkpoint_dir/MyModel-1000'

# Use graph.get_tensor_by_name() to operate the saved model

In [2]:
sess = tf.Session()
saver=tf.train.import_meta_graph('./checkpoint_dir/MyModel-1000.meta')
saver.restore(sess, tf.train.latest_checkpoint('./checkpoint_dir'))

graph = tf.get_default_graph()
w1=graph.get_tensor_by_name('w1:0')
w2=graph.get_tensor_by_name('w2:0')
feed_dict={w1:13.0,w2:17.0}

op_to_restore=graph.get_tensor_by_name('op_to_restore:0')

INFO:tensorflow:Restoring parameters from ./checkpoint_dir\MyModel-1000


# if you want to add either op or layers to re-train the model

In [5]:
import tensorflow as tf

sess = tf.Session()
# 先加载图和变量
saver = tf.train.import_meta_graph('./checkpoint_dir/MyModel-1000.meta')
saver.restore(sess, tf.train.latest_checkpoint('./checkpoint_dir'))

# 访问placeholders变量，并且创建feed-dict来作为placeholders的新值
graph = tf.get_default_graph()
w1 = graph.get_tensor_by_name("w1:0")
w2 = graph.get_tensor_by_name("w2:0")
feed_dict = {w1: 13.0, w2: 17.0}

#接下来，访问你想要执行的op
op_to_restore = graph.get_tensor_by_name("op_to_restore:0")

# 在当前图中能够加入op
add_on_op = tf.multiply(op_to_restore, 2)

print (sess.run(add_on_op, feed_dict))


INFO:tensorflow:Restoring parameters from ./checkpoint_dir\MyModel-1000
120.0


# If you wanna restore part of the model and add other op for fine-tuning, so justget op through graph.get_tensor_by_name(), and establish graph based on it.

In [21]:

saver = tf.train.import_meta_graph('vgg.meta')
# 访问图
graph = tf.get_default_graph() 
 
#访问用于fine-tuning的output
fc7= graph.get_tensor_by_name('fc7:0')
 
#如果你想修改最后一层梯度，需要如下
fc7 = tf.stop_gradient(fc7) # It's an identity function
fc7_shape= fc7.get_shape().as_list()

new_outputs=2
weights = tf.Variable(tf.truncated_normal([fc7_shape[3], num_outputs], stddev=0.05))
biases = tf.Variable(tf.constant(0.05, shape=[num_outputs]))
output = tf.matmul(fc7, weights) + biases
pred = tf.nn.softmax(output)

# Now, you run this with fine-tuning data in sess.run()


OSError: File vgg.meta does not exist.